# Image Processing

## Required Libraries

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Function Definitions

In [ ]:
def read_img(img_path):
    '''
    Read image from img_path

    Parameters
    ----------
    img_path : str
        Path of image

    Returns
    -------
        np.ndarray, shape (W, H, 3)
    '''
    img = Image.open(img_path)
    return (np.asarray(img.convert("RGB"), dtype=np.float64), img.filename)


def show_img(img_2d, caption_: str = ""):
    '''
    Show image

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3), element range [0, 255]
        The input 2D image
    caption_ : str
        (Optional) Plotting caption.
    
    Returns
    ----------
        None
    '''
    normalized = img_2d.astype(np.float64) / 255
    img = plt.imshow(normalized)
    img.axes.axis("off")
    img.axes.set_title(caption_)
    plt.show()

def brightness(img_2d: np.ndarray, factor):
    '''
    Modifying the brightness of a 2D image by scaling each pixel's component by a scalar factor

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
    factor : float
        The modifying factor.
    
    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    img_2d *= factor
    return np.clip(img_2d, 0,255, out=None)

def contrast(img_2d: np.ndarray, L=256, intensity=1):
    '''
    Modifying the contrast of a 2D image by applying 3-D histogram equalization on the image's
    intensity value
    Reference: DOI:10.1109/TIP.2010.2068555

    Parameters
    ----------
    img_2d : np.ndarray(), shape (W, H, 3)
    L : float
        Number of intensity level (higher for better seperation, i.e better contrast), 
    intensity : float
        Blending amount, range [0,1]

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    intensity = np.clip(intensity, 0, 1)
    L = np.max([0, int(L)])
    n_shape = img_2d.shape[0] * img_2d.shape[1]
    pdf = np.zeros(L, dtype=np.float32)
    single_prob = 1/n_shape
    
    # ??? calculating means for each pixels as intensity, then round it and counts for pdf
    n_in_lut = np.round((np.sum(img_2d, axis=2)/img_2d.shape[2]), 0).astype(np.int64).clip(0,L-1)
    bins, counts = np.unique(n_in_lut, return_counts=True)
    for bin in range(len(bins)):
        pdf[bins[bin]] = counts[bin]*single_prob
    c_in = np.cumsum(pdf)

    # evil mapping from luminance intensity to WxHxC enhance ratio
    f_map = np.copy(n_in_lut).reshape(n_in_lut.shape + (1,)).repeat(img_2d.shape[2],axis=2) 
    f_map = (255-(c_in[f_map[:,:,]]*L))/(255-f_map[:,:,])
    res = np.array(img_2d, copy=True)
    # messy, but this basically move color space from rgb -> cmy, then perform histogram equalization of L bins, and linear interpolate
    # between the initial image and the new enchaned value, complexity bottleneck mainly lies at building the intensity ratio f_map
    return (((255-f_map*(255-res))-res)*intensity+res).clip(0,255)
    
def flip(img_2d: np.ndarray, axis=0):
    '''
    Flipping the image around an axis

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    axis : 0 or 1
        Flipping axis. 0 for x axis, 1 for y axis

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    if(axis != 0 and axis != 1):
        print("Unknown axis")
        return img_2d
    return np.flip(img_2d, axis)

# 0: grayscale, 1: sepia
def grayscale(img_2d: np.ndarray, grayscale_mode=0):
    '''
    Transforming the original image into a grayscale or sepia result

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    grayscale_mode : 0 or 1
        Grayscale mode. 0 for traditional gray, 1 for sepia

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    # TU-R 601-2 luma transform
    grayscale_transform_coefficients = np.array([0.299, 0.587, 0.114])
    sepia_transform_coefficients = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ]).T

    if grayscale_mode == 0:
        return np.tile(np.dot(img_2d[...,:], grayscale_transform_coefficients)[...,None], (1,1,img_2d.shape[2]))
    else:
        return np.dot(img_2d[...,:], sepia_transform_coefficients).clip(0,255)
       

# Fast Almost-Gaussian Filtering - Peter Kovesi
# n_pass should be <= 6 for best result
def gaussian_blur(img_2d: np.ndarray, std_dev=1, n_pass=3, intensity=1):
    '''
    Perform aproximative Gaussian filtering on the image, with adaptive kernel size

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    std_dev : float
        The ideal Gaussian distribution function standard deviation
    n_pass : int
        The number of averaging filter pass to perform, a good aproximation requires n_pass >=  3
    intensity : float
        Blending amount, range [0,1]

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    intensity = np.clip(intensity, 0, 1)
    n_pass = int(n_pass)
    # computing integral image (Summed area tables) and ideal kernel size
    w_ideal = int(np.round(np.sqrt((12*np.power(std_dev, 2))/n_pass + 1)))
    wl = w_ideal-1 if (w_ideal % 2 == 0) else w_ideal
    wu = wl+2
    m_pass = int(np.round((12*np.power(std_dev, 2)-n_pass*np.power(wl, 2)-4*n_pass*wl-3*n_pass)/(-4*wl-4)))

    res = np.copy(img_2d)
    def iter_pass(n_c, width, out):
        r = int((width-1)/2) # guaranteed to be a whole integer
        lut_a = np.indices(out.shape[:2])
        lut_b = np.copy(lut_a)
        lut_b[1,:,:] += width
        lut_c = np.copy(lut_a) + width
        lut_d = np.copy(lut_a)
        lut_d[0,:,:] += width
        for _ in range(n_c):
            # summed area table, a 0-padding for left and top column/row to satisfy area
            avg_filter = np.pad(np.pad(out, ((r, r), (r, r), (0, 0)), mode='reflect'), \
                                ((1,0), (1,0), (0,0)), mode='constant', constant_values=0).cumsum(axis=0).cumsum(axis=1)
            # same as the below snippet but utilizing NumPy's and hardware vector acceleration, achieving x8, x9 faster results
            out = (avg_filter[lut_c[0], lut_c[1]] - avg_filter[lut_b[0], lut_b[1]] - avg_filter[lut_d[0], lut_d[1]] + avg_filter[lut_a[0], lut_a[1]]) / (width*width)
            """
            leaving here for historial reason lol - jonshung
            for x, y in np.ndindex(res.shape[:2]):
                filter_points = \
                {
                    'a': avg_filter[x,y], 
                    'b': avg_filter[x, y+(width)], 
                    'c': avg_filter[x+(width), y+(width)], 
                    'd': avg_filter[x+(width), y]
                }
                sums = filter_points['c'] - filter_points['b'] - filter_points['d'] + filter_points['a'] # average filter
                res[x, y] = sums / (width*width)
            """
        return out
            
    # m pass for wl
    res = iter_pass(m_pass, wl, res)
    # n - m pass for wu
    res = iter_pass(n_pass-m_pass, wu, res)
    return (res-img_2d)*intensity+img_2d

def um_sharpening(img_2d: np.ndarray, amount=5, intensity=1): #weird name
    '''
    Perform high-pass filter to obtain edge features and boost the result while
    adding it to the original to enhanced edges separation

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    amount : float
        The sharpening amount, usually 0.5-0.8 is good enough
    intensity : float
        blending amount, range [0,1]

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    intensity = np.clip(intensity, 0, 1)
    return ((img_2d - gaussian_blur(img_2d, std_dev=1, n_pass=3))*amount*intensity + img_2d).clip(0,255)

def bilinear_filtering(img_2d: np.ndarray, new_w, new_h):
    '''
    Bilinear filtering using simple adaptive ratio between old and new image

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    new_w : int
        New image's width
    new_h : int
        New image's height

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    old_w, old_h, channel = img_2d.shape[0], img_2d.shape[1], img_2d.shape[2]
    x_ratio = float(old_w - 1) / (new_w - 1) if new_w > 1 else 0
    y_ratio = float(old_h - 1) / (new_h - 1) if new_h > 1 else 0

    img_2d = img_2d.reshape(old_w*old_h, channel)
    X, Y = np.divmod(np.arange(new_w * new_h), new_h)
    x_l = np.floor(x_ratio * X).astype('int32')
    y_l = np.floor(y_ratio * Y).astype('int32')

    x_u = np.ceil(x_ratio * X).astype('int32')
    y_u = np.ceil(y_ratio * Y).astype('int32')
    x_weight = (x_ratio * X) - x_l
    x_weight = x_weight.reshape(x_weight.shape + (1,)).repeat(channel, axis=len(x_weight.shape))
    y_weight = (y_ratio * Y) - y_l
    y_weight = y_weight.reshape(y_weight.shape + (1,)).repeat(channel, axis=len(y_weight.shape))
    a = img_2d[x_l * old_h + y_l]
    b = img_2d[x_l * old_h + y_u]
    c = img_2d[x_u * old_h + y_l]
    d = img_2d[x_u * old_h + y_u]

    res = a * (1 - x_weight) * (1 - y_weight) + \
                c * x_weight * (1 - y_weight) + \
                b * y_weight * (1 - x_weight) + \
                d * x_weight * y_weight
    
    return res.reshape(new_w, new_h, channel)

def center_square_cut_zoom(img_2d: np.ndarray, radius=5):
    '''
    perform square-based cutting from image's center

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    radius : int
        the shortest distance from image's center to the bounding cut zone's edge
        = 1/2 side

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    radius = int(min(radius, min(img_2d.shape[0]/2, img_2d.shape[1]/2)))
    filter_rad = (radius*2)/min(img_2d.shape[0], img_2d.shape[1])
    if(filter_rad <= 1):
        filter_rad = 0
    sliced = img_2d[int(img_2d.shape[0]/2-radius):int(img_2d.shape[0]/2+radius), int(img_2d.shape[1]/2-radius):int(img_2d.shape[1]/2+radius)]
    return bilinear_filtering(sliced, img_2d.shape[0], img_2d.shape[1])

def center_circle_mask(img_2d: np.ndarray, radius=1/2):
    '''
    Masking a centered circular let-through area of the image, preserving original size

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    radius : int or float
        The masking radius, if radius is < 1, it is calculated by radius*min(W, H)

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    if(radius < 1):
        radius = int(min(img_2d.shape[0]*radius, img_2d.shape[1]*radius))
    centre = (int(img_2d.shape[0]/2), int(img_2d.shape[1]/2))
    X, Y = np.ogrid[:img_2d.shape[0], :img_2d.shape[1]]
    dist = np.sqrt((X - centre[0])**2 + (Y-centre[1])**2)
    mask = dist <= radius
    return img_2d*mask.reshape(mask.shape +(1,)).repeat(img_2d.shape[2], axis=2)

def center_double_ellipse_mask(img_2d: np.ndarray, a=512, b=64):
    '''
    Masking double 45-degree rotated symmetrical ellipses through the verticle axis

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    a : int
        the ellipse 1/2 length
    b : int
        the ellipse 1/2 width

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    a = int(a); b = int(b)
    centre = (int(img_2d.shape[0]/2), int(img_2d.shape[1]/2))
    X, Y = np.ogrid[:img_2d.shape[0], :img_2d.shape[1]]
    r1 = np.pi/4
    r2 = -np.pi/4
    dist1 = (((X-centre[0])*np.cos(r1)+(Y-centre[1])*np.sin(r1))**2)/(a)**2 + (((X-centre[0])*np.sin(r1)-(Y-centre[1])*np.cos(r1))**2)/(b)**2
    dist2 = (((X-centre[0])*np.cos(r2)+(Y-centre[1])*np.sin(r2))**2)/(a)**2 + (((X-centre[0])*np.sin(r2)-(Y-centre[1])*np.cos(r2))**2)/(b)**2
    mask = np.logical_or(dist1 <= 1, dist2 <= 1)
    return img_2d*mask.reshape(mask.shape +(1,)).repeat(img_2d.shape[2], axis=2)

def resize(img_2d: np.ndarray, factor=1):
    '''
    Resizing the image by a factor and perform bilinear interpolation if factor > 1

    Parameters
    ----------
    img_2d : np.ndarray, shape (W, H, 3)
        The input 2D image
    factor:  float
        Resizing factor, < 1 will downscale the original image, > 1 will 
        upscale the original image

    Returns
    ----------
        np.ndarray, shape (W, H, 3)
    '''
    img_2d = np.copy(img_2d)
    if factor <= 1:
        return img_2d[::int(1/factor), ::int(1/factor)]
    return bilinear_filtering(img_2d, img_2d.shape[0]*int(factor), img_2d.shape[1]*int(factor))
    

## tests

In [ ]:
def test():
    (org, org_name) = read_img(img_path='lena.png')
    mask_size = 325#1.5754*size. Magic number - jonshung
    changed = gaussian_blur(org, std_dev=0, n_pass=100, intensity=1)
    show_img(changed)
#test()



## Main FUNCTION

In [ ]:
'''
Trampoline stubs, used to provide easy user interfaced with pre-configured settings

Parameters
----------
    img_2d: np.ndarray((WxHxC))
Returns
----------
    result data: dict{ <function_name>: [<function result>: np.ndarray, ...args] }
    '''
def brightness_stub(img_2d: np.ndarray):
    correction = float(input("[Brightness] Specify the addition correcting amount (i.e. 0.2 to increase brightness by 20%)"))
    return {'brightness': [brightness(img_2d, correction+1), correction]}

def contrast_stub(img_2d: np.ndarray):
    correction = float(input("[Contrast] Specify the addition correcting amount (i.e. 0.2 to increase contrast by 20%)"))
    intensity = float(input("[Contrast] Specify the blending amount 0. - 1. (0 for no blending, 1 for fully modified image)"))
    return {'contrast': [contrast(img_2d, 256*(1+correction), intensity), correction, intensity]}

def flip_stub(img_2d: np.ndarray):
    axis = np.clip(int(input("[Flip] Specify the flipping axis (0: flip by the horizontal axis, 1: flip by the verticle axis)")), 0, 1)
    return {('flip_x' if axis==0 else 'flip_y'): [flip(img_2d, axis), axis]}

def grayscale_stub(img_2d: np.ndarray):
    gray_mode = np.clip(int(input("[Grayscale] Specify the grayscale mode (0: traditional black and white, 1: sepia gray)")), 0, 1)
    return {('grayscale' if gray_mode == 0 else 'sepia'): [grayscale(img_2d, gray_mode), gray_mode]}

def smoothen_stub(img_2d: np.ndarray):
    smoothing_amount = np.max([0, float(input("[Smoothing] Specify the smoothening amount, for amount >= 1, the image will be smoothen, \
                                          for amount < 1, the image will be sharpen when amount -> 0"))])
    intensity = float(input("[Smoothing] Specify the blending amount 0. - 1. (0 for no blending, 1 for fully modified image)"))
    if(smoothing_amount < 1):
        return {'sharpening': [um_sharpening(img_2d, 3*(1-smoothing_amount), intensity), smoothing_amount, intensity]}
    else:
        return {'blur': [gaussian_blur(img_2d, std_dev=smoothing_amount, n_pass=5, intensity=intensity), smoothing_amount, intensity]}

def center_square_cut_stub(img_2d: np.ndarray):
    rad = int(input("[Center Square Cut] Specify the cutting square radius tangent to its side"))
    return {'center_square_cut': [center_square_cut_zoom(img_2d, rad), rad]}

def mask_cut_stub(img_2d: np.ndarray):
    mask_option = np.clip(int(input("[Masking] Specify the masking option (0: circular, 1: overlapping ellipse)")), 0, 1)
    mask_rad = 0
    if(mask_option == 0):
        mask_rad = input("[Masking] Specify the circular masking radius, default: 1/2")
        mask_rad = float(mask_rad) if mask_rad != '' else 1/2
        return {"circular_mask": [center_circle_mask(img_2d, mask_rad), mask_rad]}
    else:
        a = min(img_2d.shape[0], img_2d.shape[1])/1.5754
        b = a/1.8
        return {'overlap_ellipse': [center_double_ellipse_mask(img_2d, a, b)]}
    
def resize_stub(img_2d: np.ndarray):
    size_factor = np.max([0.0, float(input("[Resize] Specify the scaling factor (downscaling for amount < 1, upscaling for amount > 1)"))])
    return {('resize_up' if size_factor >= 1 else 'resize_down'): [resize(img_2d, size_factor), size_factor]}
    
def process_routine(img_2d: np.ndarray):
    dr = dict()
    for k, v in fn_map.items():
        if(k == '0'):
            continue
        dr.update(v(img_2d))
    return dr

fn_map: dict = {
    '0': process_routine,
    '1': brightness_stub,
    '2': contrast_stub,
    '3': flip_stub,
    '4': grayscale_stub,
    '5': smoothen_stub,
    '6': center_square_cut_stub,
    '7': mask_cut_stub,
    '8': resize_stub
}

'''
Save an image, at the current running directory

Parameters
----------
    img_2d: np.ndarray((WxHxC))
        Image to save
    img_name: str
        Saving name
'''
def save_img(img_2d: np.ndarray, img_name):
    Image.fromarray(np.uint8(img_2d), "RGB").save(img_name + ".png")

'''
Program main entry point
'''
def main():
    img_path = input("Enter image path: ")
    org, org_name = read_img(img_path=img_path)
    fn = fn_map.get(input("Select image processing routine\n0: perform all routine once\n1: brightness correction\n\
                          2: contrast correction\n3: flip image\n4: grayscale conversion\n5: smoothing/sharpening routine\n\
                          6: center square cut and zoom\n7: circular/overlap ellipse masking\n8: resize image\n"), fn_map['0'])
    res = fn(org)
    ret = "Result:\n"
    for r, e in res.items():
        ret += "[" + r + "] " + "{" + ', '.join(str(i) for i in e[1:]) + "}\n"
        show_img(e[0])
    ret += "Save?(y/n)"
    c = input(ret).lower()  
    if(c == 'y' or len(c) == 0):
        filename = org_name.split('.')[0]
        for k, v in res.items():
            save_img(v[0], '_'.join([filename, k]))

In [ ]:
# Call main function
main()